# Prolog to Query Databases

Prolog is a programming language that relies on relations. It is used to derive facts and rules based on those relations. Widely used in different fields, such as Artificial Intelligence and Computational Linguistics, and is based upon first-order and formal logic. In this tutorial we'll first explore the basic aspcects of Prolog, such as linking a database and simple queries, and then move on to more complicated subjects, such as joining tables, filtering, updating, ...

This tutorial will be a good foundation on how to use Prolog if you're a beginner on the subject and if you're already familiar with Prolog, you can use it to have a little reminder on the basics.

### SWI-Prolog

First, let's make sure that you have [SWI-Prolog](https://www.swi-prolog.org) installed on your machine by running the following commands. This will download the SWI-Prolog, which is a free and portable Prolog environment that is easy to use and compliant with textbooks, using the PPA (Ubunut Personnal Package Archive). 

If you're not a Ubuntu user, we strongly advise you to use the following Notebook in [Google Colab](https://colab.research.google.com).

![](https://www.swi-prolog.org/icons/swipl.png)

In [1]:
# !apt-get install software-properties-common
# !apt-add-repository ppa:swi-prolog/stable
# !apt-get update
# !apt-get install swi-prolog

### Library

In this tutorial, we'll use a package to allow us to query databases in Python by using the `SWI-Prolog` environment. This package is named `PySWIP` and can be installed with the following command :

In [2]:
# # install from the Github repo since the PyPi package produces errors

# !pip install git+https://github.com/yuce/pyswip@master#egg=pyswip

  Cloning https://github.com/yuce/pyswip (to revision master) to /private/var/folders/d5/xjnfc_z96714df80fs1j8sg80000gn/T/pip-install-jqu0msv0/pyswip_7b44d5e25d534b6a9bac59a800d2b58d
  Running command git clone --filter=blob:none --quiet https://github.com/yuce/pyswip /private/var/folders/d5/xjnfc_z96714df80fs1j8sg80000gn/T/pip-install-jqu0msv0/pyswip_7b44d5e25d534b6a9bac59a800d2b58d
  Resolved https://github.com/yuce/pyswip to commit 59016e0841f56177d1b18ec08fd9b67792bd0a97
  Preparing metadata (setup.py) ... done
  Created wheel for pyswip: filename=pyswip-0.2.11-py3-none-any.whl size=28436 sha256=55c9c3013d796ebd77fe1683d473eca8b2af50c19708f7229f28545646e46694
  Stored in directory: /private/var/folders/d5/xjnfc_z96714df80fs1j8sg80000gn/T/pip-ephem-wheel-cache-9ajz90af/wheels/cc/f6/91/c3240f7b687960b33b5b5e93fadc20c8391d71d0126b0336e2
Successfully built pyswip
  Attempting uninstall: pyswip
    Found existing installation: pyswip 0.2.10
    Uninstalling pyswip-0.2.10:
      Successf

### Prolog in PySWIP

With the `PySWIP` package installed, we can create a `Prolog` object that will allow us to query with Prolog and use the `assertz()` function to create facts about the instances in our database. 

In [3]:
from pyswip import *

prolog = Prolog()

prolog.assertz("father(john, jim)")
prolog.assertz("mother(jane, jim)")

prolog.assertz("father(john, jenny)")
prolog.assertz("mother(jane, jenny)")

prolog.assertz("father(john, jack)")
prolog.assertz("father(mary, jack)")

prolog.assertz("father(rob, andrew)")
prolog.assertz("mother(mary, andrew)")

prolog.assertz("father(bob, billy)")
prolog.assertz("mother(sue, billy)")

### Prolog basics

In Prolog, everything is based on facts and rules. Facts are specified in the format as seen in the cell above and represent informations known about our dataset. We can deduce new relations between instances using the rules (by doing queries).

A query is represented by a fact followed by the parameters between parantheses followed by `:-` the part of the query where the rule is formulated.

- `,` represents the logical AND operator

- Writing two queries for the same fact will be interpreted as a logical OR operator

- `.` delimits the end of a query

- `\+` represents the logical NOT operator

- `[{}]` represents True

- `[]` represents False


Simples queries over the facts will simply return True or False, but we can go further by doing variable queries.

When using pyswip we can use the `query()` function to query the database and get the results. The results are returned as a list of dictionaries, where each dictionary represents a result and the keys are the variables used in the query. But we still need to explicitly use the `list()` function to get the results.

We can create a small function that can simply check if a query returns True or False and can start checking some facts about the relations we've just created :

In [4]:
# How to do a simple query

def check_query(query):
    query = list(query)
    return len(query) != 0

print(check_query(prolog.query("father(john, jim)")))
print(check_query(prolog.query("mother(sue, jim)")))

True
False


In [5]:
# How to do a variable query

def print_query(query, field):
    query = list(query)
    for result in query:
        print(result[field])

jims_father = prolog.query("father(X, jim)")

print("Jim's father :")
print_query(jims_father, 'X')

Jim's father :
john


In [6]:
rule_parent1 = "parent(X, Y) :- father(X, Y)"
rule_parent2 = "parent(X, Y) :- mother(X, Y)"

prolog.assertz(rule_parent1)
prolog.assertz(rule_parent2)

We can now create our own rule to query this small database to look for siblings :

In [7]:
# Rule :
rule_sibling = "sibling(X, Y) :- father(F, X), father(F, Y), mother(M, X), mother(M, Y), X \= Y"

# Assert the rule
prolog.assertz(rule_sibling)

# Check the rule
jims_siblings = prolog.query("sibling(X, jim)")

print("Jim's siblings :")
print_query(jims_siblings, 'X')

Jim's siblings :
jenny


Can you now come up with a rule to check for stepsiblings ?

In [8]:
# Rules :
rule_step_sibling = "step_sibling(X, Y) :- parent(P, X), parent(P, Y), parent(Q, X), parent(R, Y), P \= Q, P \= R, Q \= R"

# Assertion
prolog.assertz(rule_step_sibling)

# Check the rules
jacks_step_siblings = prolog.query("step_sibling(X, jack)")
print("Jack's step siblings :")
print_query(jacks_step_siblings, 'X')

Jack's step siblings :
jim
jenny
andrew


### Consult

To avoid using the `assertz()` function to create facts, we can use the `consult()` function to load a file containing the facts. This file must be a `.pl` file :

In [9]:
animal = Prolog()
animal.consult("family.pl")

result = animal.query("grandparent(X, Y)")
print(list(result))

[{'X': 'john', 'Y': 'ann'}, {'X': 'john', 'Y': 'bob'}, {'X': 'john', 'Y': 'jim'}]


## Declarative Programming

In Prolog, we can use declarative programming to define the rules of a program. The rules are defined in the form of facts and rules. The facts are the statements that are true. The rules are the statements that are true if the conditions are met.
This is a way to solve problem by telling what a program should accomplish, rather than specifying how it should be done. 

In [10]:
sum = Prolog()
sum.consult("sum.pl")

result = sum.query("sum_list([1, 2, 3, 4, 5], Sum)")

print_query(result, 'Sum')

15


This code is defining a predicate sum_list in Prolog, which calculates the sum of a list of numbers.

The predicate takes two arguments: 
 - the first argument is the list of numbers
 - the second argument is the sum of those numbers.

The first line of the code `sum_list([], 0)` is a base case that defines the sum of an empty list as 0.

The second line of the code `sum_list([Head|Tail], Sum)` defines the recursive case, where `Head` is the first element of the list and `Tail` is the rest of the list.

The recursive case then calls the `sum_list` predicate again with the Tail argument to calculate the sum of the remaining elements of the list, and stores that sum in the TailSum variable.

Finally, the sum of Head and TailSum is computed using the `is` operator, and the result is stored in the `Sum` variable. This Sum value is returned as the final answer of the `sum_list` predicate.

Overall, the `sum_list` predicate recursively computes the sum of a list of numbers by breaking it down into smaller subproblems (i.e., the sum of the tail of the list), and then combining the results to get the final answer.

In [11]:
meta = Prolog()
meta.consult("meta.pl")


result = meta.query("atoms_to_preds([foo, bar, baz], Preds).")
print(list(result))

[{'Preds': [',(foo_1, foo_2)', ',(bar_1, bar_2)', ',(baz_1, baz_2)']}]


This code defines a predicate `atoms_to_preds` that takes a list of atoms and returns a list of two-argument predicates with those names. Each predicate takes two arguments of any type and simply returns those arguments as a tuple.

To accomplish this, `atoms_to_preds` uses several meta-predicates. First, it constructs the names of the predicates using atom_concat and then converts those names to actual predicate terms using `atom_to_term`. It then combines these predicates into two-argument tuples using the Prolog tuple syntax.

Finally, the code uses the Prolog query system to demonstrate how atoms_to_preds can be used to create a list of predicates from a list of atoms. The query `atoms_to_preds([foo, bar, baz], Preds)` returns the list `[(foo_1, foo_2), (bar_1, bar_2), (baz_1, baz_2)]`, which demonstrates how meta-programming can be used to dynamically generate code.